In [4]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from huggingface_hub import HfApi

### Uploading expl

In [2]:
task_name = "cosqa"

In [10]:
expl_df = pd.read_csv(cosqa_expl_path)
expl_df.head()

,Unnamed: 0,query_id,doc,corpus_id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_deepseek_1_cleaned,explanation_deepseek_2_cleaned,explanation_deepseek_3_cleaned,explanation_deepseek_4_cleaned
0,0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe provided code is a Python method named `...,"\n1. `def writeBoolean(self, n):` - This line ...",\nThis code snippet is a function that writes ...,\nThe code snippet provided is a method named ...,The doc string and code both write the boolean...,"Alright, I need to break down this Python code...",The code snippet writes a boolean value to a s...,The code snippet defines a method called write...
1,1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThe code snippet provided is a Python functi...,The paste function pastes data from the clipbo...,The code snippet begins by defining a function...,The code snippet is a function called paste th...,The code snippet defines a function called pas...
2,2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...","\nThis Python function, `_format_json`, takes ...","\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\n```python\nimport json\nimport sys\nimport p...,The docstring explains that the function _form...,"```pythondef _format_json(data, theme): """"""...",This code snippet is a function called _format...,To explain the code snippet in a way that can ...
3,3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...","\nThe provided Python function, `create_path(p...",\n1. The code snippet defines a function named...,\nThis code snippet helps you create a new dir...,\nThe provided code snippet is a Python functi...,The doc and code create an absolute path from ...,"Okay, I need to explain this Python code snipp...",This code creates a file system where any path...,The code snippet defines a function called cre...
4,4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,\nThis code snippet defines a function called ...,\nThis code snippet is a function that convert...,\nThe code snippet provided is a function defi...,The code checks if an object `x` is an array o...,"Alright, so I've got this Python code snippet ...",The code snippet is a Python function called _...,"Okay, I'm going to try to figure out how to ex..."


In [3]:
queries_corpus_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-queries-corpus")
qrels_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-qrels")
corpus_data = queries_corpus_dataset['corpus']
query_data = queries_corpus_dataset['queries']
qrels_data_test = qrels_dataset['test']          
qrels_data_train = qrels_dataset['train']
qrels_data_valid = qrels_dataset['valid']

In [4]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'

def add_expl(corpus_data, explanation_df_path, col_name='explanation_deepseek_1_cleaned'):
    expl_df = pd.read_csv(explanation_df_path)
    
    # Make sure column names are consistent
    expl_df.rename(columns={"corpus_id": "corpus-id"}, inplace=True)

    # Create a map of corpus-id to explanation
    expl_map = {
        str(row["corpus-id"]): row[col_name]
        for _, row in expl_df.iterrows()
        if pd.notna(row[col_name]) and str(row[col_name]).strip()
    }

    # Replace corpus_data['text'] with explanation where _id matches
    corpus_list = corpus_data.to_list()

    for item in corpus_list:
        corpus_id = item["_id"]
        if corpus_id in expl_map:
            item["text"] = expl_map[corpus_id]

    # Filter out entries with empty or missing text
    corpus_list = [doc for doc in corpus_list if doc.get("text", "").strip()]

    # Convert back to Dataset
    corpus_data_with_expl = Dataset.from_list(corpus_list).cast(corpus_data.features)

    return corpus_data_with_expl


In [6]:
corpus_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [7]:
corpus_data['text'][0]

'def writeBoolean(self, n):\n        """\n        Writes a Boolean to the stream.\n        """\n        t = TYPE_BOOL_TRUE\n\n        if n is False:\n            t = TYPE_BOOL_FALSE\n\n        self.stream.write(t)'

In [8]:
corpus_data = add_expl(corpus_data, cosqa_expl_path, "explanation_granite_1")

Casting the dataset: 100%|██████████| 20604/20604 [00:00<00:00, 1600745.36 examples/s]


In [9]:
corpus_data['text'][0]

"\nThe provided code is a Python method named `writeBoolean` that writes a Boolean value to a stream. It takes one argument `n`, which is a Boolean value. If `n` is `False`, it sets the value of `t` to `TYPE_BOOL_FALSE`. Otherwise, it sets `t` to `TYPE_BOOL_TRUE`. Finally, it writes the value of `t` to the stream using the `write` method of the `self.stream` object. The doc string provides a brief description of the method's purpose and functionality."

In [11]:
corpus_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [12]:
query_data

Dataset({
    features: ['_id', 'partition', 'text', 'title', 'language', 'meta_information'],
    num_rows: 20604
})

In [13]:
qrels_data_test

Dataset({
    features: ['query_id', 'corpus_id', 'score'],
    num_rows: 500
})

In [14]:
corpus_data.push_to_hub("vaishnavisha/corpus_data_cosqa_granite1")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/corpus_data_cosqa_granite1/commit/b7083027e34998b57eb6650f55877d0ce8e1eca7', commit_message='Upload dataset', commit_description='', oid='b7083027e34998b57eb6650f55877d0ce8e1eca7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/corpus_data_cosqa_granite1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/corpus_data_cosqa_granite1'), pr_revision=None, pr_num=None)

### Uploading results

In [16]:
df = pd.read_parquet('/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/cosqa/granite1/dres/intfloat_e5-base-v2/augmented_retrieval_analysis.parquet')

In [17]:
dataset = Dataset.from_pandas(df)
dataset.push_to_hub("vaishnavisha/retrieved_results_cosqa_granite1_dres_intfloat_e5-base-v2")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.25s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/vaishnavisha/retrieved_results_cosqa_granite1_dres_intfloat_e5-base-v2/commit/3b656d6f7b7e9a6a1a46ef49b37b87e35f564e1a', commit_message='Upload dataset', commit_description='', oid='3b656d6f7b7e9a6a1a46ef49b37b87e35f564e1a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vaishnavisha/retrieved_results_cosqa_granite1_dres_intfloat_e5-base-v2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vaishnavisha/retrieved_results_cosqa_granite1_dres_intfloat_e5-base-v2'), pr_revision=None, pr_num=None)